<a href="https://colab.research.google.com/github/otitamario/sp-pa-gep/blob/main/experiments/Example5_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Clone the repository into Colab runtime
!git clone https://github.com/otitamario/sp-pa-gep.git

# Move into repo root
%cd sp-pa-gep

# Make sure Python sees the project root
import sys
sys.path.append(".")

Cloning into 'sp-pa-gep'...
remote: Enumerating objects: 286, done.
remote: Counting objects: 100% (102/102), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 286 (delta 71), reused 23 (delta 23), pack-reused 184 (from 1)
Receiving objects: 100% (286/286), 4.39 MiB | 12.26 MiB/s, done.
Resolving deltas: 100% (139/139), done.
/content/sp-pa-gep


In [2]:
import os
# =========================
# OUTPUT DIRECTORY (save plots here)
# =========================
FIGDIR = "figures"
os.makedirs(FIGDIR, exist_ok=True)

In [4]:
import matplotlib.pyplot as plt

def plot_metric_across_cases(logs_by_case, metric, ylabel, title, outpath):
    plt.figure()
    for name, logs in logs_by_case.items():
        if metric == "step":
            y = [L.step for L in logs]
        elif metric == "residual":
            y = [L.residual for L in logs if L.residual is not None]
        elif metric == "error":
            y = [L.error for L in logs if L.error is not None]
        else:
            continue

        if len(y) == 0:
            continue

        linestyle = "-" if "SPPA" in name else "--"
        plt.semilogy(range(len(y)), y, linestyle=linestyle, linewidth=2, label=name)

    plt.xlabel("Iteration")
    plt.ylabel(ylabel)
    plt.title(title)
    plt.grid(True, which="both", linestyle="--", linewidth=0.5)
    plt.legend(frameon=False)
    plt.tight_layout()
    plt.savefig(outpath, dpi=300)
    plt.close()

In [5]:
import numpy as np
import src.benchkit as bk  # if you saved the library; otherwise paste needed functions
# if you pasted plot_metric_across_cases into notebook, import it directly

n = 100
x_star = np.zeros(n)
x0 = np.ones(n)
u = np.zeros(n)
tau = 1e-12  # early stopping threshold on step size

N = 200
alpha_seq = np.array([1.0/(k+2) for k in range(N)], float)
r = 1.0
mu_values = [0.1, 1.0, 5.0]

def resolvent_strong_monotone(mu):
    def S(x, r):
        return x / (1.0 + r*mu)
    return S

def make_sppa_step(mu):
    S = resolvent_strong_monotone(mu)
    def step_fn(x, k):
        alpha = float(alpha_seq[k])
        u_k = S(x, r)
        x_new = alpha*u + (1.0-alpha)*u_k
        return x_new, {}
    return step_fn

def make_wppa_step(mu):
    S = resolvent_strong_monotone(mu)
    def step_fn(x, k):
        alpha = float(alpha_seq[k])
        u_k = S(x, r)
        x_new = alpha*x + (1.0-alpha)*u_k
        return x_new, {}
    return step_fn

def make_residual(mu):
    S = resolvent_strong_monotone(mu)
    def residual_fn(x):
        return float(np.linalg.norm(x - S(x, r)))
    return residual_fn

def error_fn(x):
    return float(np.linalg.norm(x - x_star))

# --- run all cases ---
logs_by_case = {}
summaries = []

for mu in mu_values:
    logs_sppa, sum_sppa = bk.run(
        method_name=f"SPPA μ={mu}",
        x0=x0,
        max_iter=N,
        stop_tol_step=tau,   # keep full length for fair plot
        step_fn=make_sppa_step(mu),
        residual_fn=make_residual(mu),
        error_fn=error_fn,
    )
    logs_wppa, sum_wppa = bk.run(
        method_name=f"WPPA μ={mu}",
        x0=x0,
        max_iter=N,
        stop_tol_step=tau,
        step_fn=make_wppa_step(mu),
        residual_fn=make_residual(mu),
        error_fn=error_fn,
    )
    # --- empirical linear rate estimation (from error) ---
    k0 = 10  # ignore transient
    for name, logs in [("SPPA", logs_sppa), ("WPPA", logs_wppa)]:
        e = np.array([L.error for L in logs if L.error is not None], float)
        if len(e) > k0 + 5:  # ensure enough data
            m, _ = np.polyfit(np.arange(k0, len(e)), np.log(e[k0:]), 1)
            q_emp = float(np.exp(m))
            q_theory = 1.0 / (1.0 + r * mu)
            print(f"{name} μ={mu}: q_emp={q_emp:.6f}, q_theory={q_theory:.6f}")

    logs_by_case[f"SPPA μ={mu}"] = logs_sppa
    logs_by_case[f"WPPA μ={mu}"] = logs_wppa
    summaries.extend([sum_sppa, sum_wppa])

# --- one figure per metric across all μ ---
plot_metric_across_cases(
    logs_by_case=logs_by_case,
    metric="step",
    ylabel=r"$\|x_{n+1}-x_n\|$",
    title=None,
    outpath="figures/ex51_steps_all_mu.png",
)

plot_metric_across_cases(
    logs_by_case=logs_by_case,
    metric="residual",
    ylabel=r"$R(x_n)$",
    title=None,
    outpath="figures/ex51_residual_all_mu.png",
)

plot_metric_across_cases(
    logs_by_case=logs_by_case,
    metric="error",
    ylabel=r"$\|x_n\|$",
    title=None,
    outpath="figures/ex51_error_all_mu.png",
)

# --- one table for all μ (6 rows) ---
print(bk.latex_tables_split(
    summaries,
    caption_perf="Computational performance for Example 5.1 (all $\\mu$).",
    label_perf="tab:ex51_perf",
    caption_acc="Accuracy summary for Example 5.1 (all $\\mu$).",
    label_acc="tab:ex51_acc",
))

SPPA μ=0.1: q_emp=0.898340, q_theory=0.909091
WPPA μ=0.1: q_emp=0.910163, q_theory=0.909091
SPPA μ=1.0: q_emp=0.479464, q_theory=0.500000
WPPA μ=1.0: q_emp=0.517554, q_theory=0.500000
SPPA μ=5.0: q_emp=0.155808, q_theory=0.166667
WPPA μ=5.0: q_emp=0.212727, q_theory=0.166667
\begin{table}[!ht]
\centering
\begin{tabular}{lrrrr}
\hline
Method & It. & Tot (s) & Avg res (s) & Step$_f$\\
\hline
SPPA μ=0.1 & 200 & 0.0135 & 0.00001 & 2.7639e-11 \\
WPPA μ=0.1 & 200 & 0.0061 & 0.00001 & 8.5072e-09 \\
SPPA μ=1.0 & 38 & 0.0025 & 0.00001 & 9.8191e-13 \\
WPPA μ=1.0 & 48 & 0.0011 & 0.00001 & 8.5265e-13 \\
SPPA μ=5.0 & 17 & 0.0004 & 0.00001 & 1.7569e-13 \\
WPPA μ=5.0 & 23 & 0.0005 & 0.00001 & 9.9385e-13 \\
\hline
\end{tabular}
\caption{Computational performance for Example 5.1 (all $\mu$).}
\label{tab:ex51_perf}
\end{table}

\begin{table}[!ht]
\centering
\begin{tabular}{lrr}
\hline
Method & $\lVert x_n-\bar{x}\rVert$ & $R(x_n)$\\
\hline
SPPA μ=0.1 & 2.6198e-10 & 2.3816e-11 \\
WPPA μ=0.1 & 8.5540e-08 

In [ ]:
# ─── 1. Configurar seu nome e email (só precisa rodar uma vez)
!git config --global user.name "otitamario"
!git config --global user.email "mario2013sj@gmail.com"

In [ ]:
!git add .

In [ ]:
# ─── 2. Método mais seguro e prático (usando cache de credencial) ────────────────

from google.colab import userdata

# Salve o token como segredo no Colab (muito recomendado!)
# Vá na barra lateral esquerda → chave de cadeado "Secrets"
# Adicione um segredo chamado: GITHUB_TOKEN
# Cole seu token lá e marque "Notebook access"

token = userdata.get('GITHUB_TOKEN')          # pega o segredo
username = "otitamario"               # seu username
repo = "sp-pa-gep"              # ex: meu-projeto

# Configura o remote com o token (https)
!git remote set-url origin https://{token}@github.com/{username}/{repo}.git

# Ou se ainda não tem remote configurado:
#!git remote add origin https://{token}@github.com/{username}/{repo}.git

In [ ]:
!git commit -m "Example 5.1"

[main dc6eff0] Example 5.1
 3 files changed, 0 insertions(+), 0 deletions(-)
 create mode 100644 figures/ex51_error_all_mu.png
 create mode 100644 figures/ex51_residual_all_mu.png
 create mode 100644 figures/ex51_steps_all_mu.png


In [ ]:
!git pull origin main --rebase

From https://github.com/otitamario/sp-pa-gep
 * branch            main       -> FETCH_HEAD
Current branch main is up to date.


In [ ]:
!git push origin main

Enumerating objects: 8, done.
Counting objects: 100% (8/8), done.
Delta compression using up to 2 threads
Compressing objects: 100% (6/6), done.
Writing objects: 100% (6/6), 574.42 KiB | 13.36 MiB/s, done.
Total 6 (delta 2), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/otitamario/sp-pa-gep.git
   d07934a..dc6eff0  main -> main
